In [2]:
#Multinomial Naive Bayes
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

data = pd.read_csv('reddit_train.csv')
dataTest = pd.read_csv('reddit_test.csv')

In [3]:
#Creating a dictionary to store subreddit names and numbers
subRedditNumber = {}

j = 0

y = data['subreddits'].tolist()

for i in range(0, len(y)):
    if y[i] in subRedditNumber.values():
        continue
    else:
        subRedditNumber[j] = y[i]
        j += 1

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

comments_total = data['comments'].tolist() + dataTest['comments'].tolist()

#Vectorizing
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(comments_total)
#Reddit Test for Kaggle
TestX = X[70000:len(comments_total)]

In [5]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import normalize

#Implementing Bagging

allpredictions = []

for i in range(25):
    X_train_loop, X_test_loop, Y_train_loop, Y_test_loop = train_test_split(X[:70000], y, test_size=0.1)
    #Normalizing
    X_train_loop = normalize(X_train_loop)
    X_test_loop = normalize(X_test_loop)
    clf_nb_loop = MultinomialNB(alpha=0.2).fit(X_train_loop, Y_train_loop)
    #Validation Test scores
    score_nb = clf_nb_loop.score(X_test_loop, Y_test_loop)
    predictions = clf_nb_loop.predict(TestX)
    allpredictions.append(predictions)
    print (score_nb)

votingPredictions = []

from collections import Counter

for i in range(30000):
    c = Counter([allpredictions[0][i], allpredictions[1][i], allpredictions[2][i], allpredictions[3][i], 
                allpredictions[4][i], allpredictions[5][i], allpredictions[6][i], allpredictions[7][i], 
                allpredictions[8][i], allpredictions[9][i], allpredictions[10][i], allpredictions[11][i], 
                allpredictions[12][i], allpredictions[13][i], allpredictions[14][i], allpredictions[15][i],
                allpredictions[16][i], allpredictions[17][i], allpredictions[18][i], allpredictions[19][i], 
                allpredictions[20][i], allpredictions[21][i], allpredictions[22][i], allpredictions[23][i], 
                allpredictions[24][i]])
    value, count = c.most_common()[0]
    votingPredictions.append(value)

0.5788571428571428
0.5687142857142857
0.5632857142857143
0.5714285714285714
0.5817142857142857
0.5771428571428572
0.5771428571428572
0.5701428571428572
0.57
0.5782857142857143
0.5752857142857143
0.573
0.5695714285714286
0.5752857142857143
0.5724285714285714
0.5684285714285714
0.5701428571428572
0.5748571428571428
0.5655714285714286
0.5728571428571428
0.5708571428571428
0.5684285714285714
0.5632857142857143
0.5707142857142857
0.5667142857142857


In [22]:
#Script for submission to Kaggle

Category = []

for i in range(0,len(votingPredictions)):
    for key, val in subRedditNumber.items():    # for name, age in dictionary.iteritems():  (for Python 2.x)
        if val == votingPredictions[i]:
            Category.append(val)

from pandas import DataFrame

Id = []

for i in range(0, len(votingPredictions)):
    Id.append(i)
    
Cars = {'Id': Id, 'Category': Category}

submissionDF = DataFrame(Cars, columns= ['Id', 'Category'])

submissionDF.head()

submissionDF.to_csv('submission_19_1.csv', index=False)